In [1]:
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime
import os
from dotenv import load_dotenv

load_dotenv()

mt5_account = int(os.getenv('MT5_ACCOUNT'))
mt5_password = os.getenv('MT5_PASSWORD')
mt5_server = os.getenv('MT5_SERVER')
mt5_path = os.getenv('MT5_PATH')

if not mt5.initialize(path=mt5_path, login=mt5_account, password=mt5_password, server=mt5_server):
    print("initialize() failed, error code =", mt5.last_error())
    quit()

print("Successfully connected to MetaTrader 5.")

symbol = "XAUUSDm"
timeframe_h4 = mt5.TIMEFRAME_H4
timeframe_h1 = mt5.TIMEFRAME_H1
start_date = datetime(2023, 1, 1)
end_date = datetime.now()

h4_rates = mt5.copy_rates_range(symbol, timeframe_h4, start_date, end_date)
df_h4 = pd.DataFrame(h4_rates)
df_h4['time'] = pd.to_datetime(df_h4['time'], unit='s')
df_h4.set_index('time', inplace=True)

h1_rates = mt5.copy_rates_range(symbol, timeframe_h1, start_date, end_date)
df_h1 = pd.DataFrame(h1_rates)
df_h1['time'] = pd.to_datetime(df_h1['time'], unit='s')
df_h1.set_index('time', inplace=True)

print(f"Fetched {len(df_h4)} H4 bars and {len(df_h1)} H1 bars for {symbol}.")

df_h4['EMA_100'] = df_h4['close'].ewm(span=100, adjust=False).mean()

delta = df_h1['close'].diff()

gain = (delta.where(delta > 0, 0)).fillna(0)
loss = (-delta.where(delta < 0, 0)).fillna(0)

avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()

rs = avg_gain / avg_loss

df_h1['RSI_14'] = 100 - (100 / (1 + rs))

print("\n--- Verification ---")
print("H4 DataFrame with EMA_100 (last 5 rows):")
print(df_h4[['open', 'high', 'low', 'close', 'EMA_100']].tail())

print("\nH1 DataFrame with RSI_14 (last 5 rows):")
print(df_h1[['open', 'high', 'low', 'close', 'RSI_14']].tail())

mt5.shutdown()
print("\nMetaTrader 5 connection shut down.")

Successfully connected to MetaTrader 5.
Fetched 4387 H4 bars and 16211 H1 bars for XAUUSDm.

--- Verification ---
H4 DataFrame with EMA_100 (last 5 rows):
                         open      high       low     close      EMA_100
time                                                                    
2025-09-26 20:00:00  3766.762  3769.530  3760.050  3761.041  3666.612205
2025-09-28 20:00:00  3764.718  3772.495  3763.347  3766.899  3668.598082
2025-09-29 00:00:00  3766.858  3799.436  3763.935  3794.340  3671.088021
2025-09-29 04:00:00  3794.355  3819.874  3792.447  3806.343  3673.766337
2025-09-29 08:00:00  3806.381  3819.751  3806.381  3815.764  3676.578172

H1 DataFrame with RSI_14 (last 5 rows):
                         open      high       low     close     RSI_14
time                                                                  
2025-09-29 06:00:00  3813.682  3819.874  3810.953  3817.400  65.757109
2025-09-29 07:00:00  3817.360  3819.347  3805.007  3806.343  63.131746
2025-09-2